In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
import os
import gc
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import ShuffleSplit

In [ ]:
import random
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ["PYTHONHASHSEED"] = str(seed)

set_all_seeds(420)

In [ ]:
model_mn2 = MobileNetV2(
    input_shape=None, alpha=1.0, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)

14536120/14536120 [==============================] - 0s 0us/step


In [ ]:
model_mn2.summary()

In [ ]:
layer_name = 'block_15_add'
feature_extractor = Model(inputs=model_mn2.input, outputs=model_mn2.get_layer(layer_name).output)

In [ ]:
plants_names = []

for plant in os.listdir('gdrive/MyDrive/flower_data/encoded/train'):
    plants_names += [plant.split('.npy')[0]]

plants_names.sort()
plant_name_to_id = {plant: i for i, plant in enumerate(plants_names)}

In [ ]:
instances = {}
labels = {}

for set_kind in ["test", "train", "val"]:
    X_per_class = []
    y_per_class = []

    for i, file in enumerate(os.listdir(f"gdrive/MyDrive/flower_data/encoded/{set_kind}")):
        print(f"\r[{i}] {set_kind}/{file}", end='')
        plants_encoded = np.load(f"gdrive/MyDrive/flower_data/encoded/{set_kind}/{file}")
        plant_name = file.split('.npy')[0]

        X_per_class += [plants_encoded]
        y_per_class += [plant_name_to_id[plant_name] for _ in range(plants_encoded.shape[0])]

    instances[set_kind] = np.concatenate(X_per_class)
    labels[set_kind] = np.array(y_per_class)

[101] val/16.npy

In [ ]:
n_classes = len(plants_names)

X_train = instances['train']
y_train = np.eye(n_classes)[labels['train']]

X_valid = instances['val']
y_valid = np.eye(n_classes)[labels['val']]

X_test = instances['test']
y_test = np.eye(n_classes)[labels['test']]

In [ ]:
input_layer = tf.keras.layers.Input(shape=[7, 7, 160], name="submodel_inputs")

block1 = tf.keras.layers.Conv2D(filters=320, kernel_size=1, use_bias=False, name="block_1_1_conv2d")(input_layer)
block1 = tf.keras.layers.BatchNormalization(name="block_1_1_bn")(block1)
block1 = tf.keras.layers.ReLU(name="block_1_1_relu")(block1)
block1 = tf.keras.layers.DepthwiseConv2D(kernel_size=3, padding='same', use_bias=False, name="block_1_2_depthwise")(block1)
block1 = tf.keras.layers.BatchNormalization(name="block_1_2_bn")(block1)
block1 = tf.keras.layers.ReLU(name="block_1_2_relu")(block1)
block1 = tf.keras.layers.Conv2D(filters=160, kernel_size=1, use_bias=False, name="block_1_3_conv2d")(block1)
block1 = tf.keras.layers.BatchNormalization(name="block_1_3_bn")(block1)
block1 = tf.keras.layers.ReLU(name="block_1_3_relu")(block1)

se_block1 = tf.keras.layers.GlobalAveragePooling2D(name="se_block_1_gap2d")(input_layer)
se_block1 = tf.keras.layers.Dense(units=24, name="se_block_1_1_dense")(se_block1)
se_block1 = tf.keras.layers.ReLU(name="se_block_1_1_relu")(se_block1)
se_block1 = tf.keras.layers.Dense(units=160, name="se_block_1_2_dense")(se_block1)
se_block1 = tf.keras.layers.Activation("sigmoid", name="se_block_1_2_sigmoid")(se_block1)

multiplication1 = tf.keras.layers.Multiply()([block1, se_block1])
addition1 = tf.keras.layers.Add()([input_layer, multiplication1])

block4 = tf.keras.layers.Conv2D(filters=720, kernel_size=1, use_bias=False, name="block_3_1_cond2d")(addition1)
block4 = tf.keras.layers.BatchNormalization(name="block_3_1_bn")(block4)
block4 = tf.keras.layers.ReLU(name="block_3_2_relu")(block4)

final_part = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling2d")(block4)
final_part = tf.keras.layers.Dropout(rate=0.2, name="dropout")(final_part)
output_layer = tf.keras.layers.Dense(units=102, activation="softmax", kernel_initializer="he_normal", name="predictions")(final_part)


model = Model(input_layer, output_layer)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"]
)

In [ ]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.66, patience=4)

In [ ]:
idx = np.arange(X_train.shape[0])
np.random.shuffle(idx)

X_train_shuffled = X_train[idx]
y_train_shuffled = y_train[idx]

# Free some precious RAM
del X_train
del instances['train']

In [ ]:
model.fit(
    X_train_shuffled, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler]
)

Epoch 1/10
333/333 [==============================] - 20s 10ms/step - loss: 1.7279 - accuracy: 0.6547 - val_loss: 0.6693 - val_accuracy: 0.8655 - lr: 0.0010
Epoch 2/10
333/333 [==============================] - 3s 9ms/step - loss: 0.3699 - accuracy: 0.9229 - val_loss: 0.3399 - val_accuracy: 0.9188 - lr: 0.0010
Epoch 3/10
333/333 [==============================] - 3s 8ms/step - loss: 0.1860 - accuracy: 0.9626 - val_loss: 0.2733 - val_accuracy: 0.9272 - lr: 0.0010
Epoch 4/10
333/333 [==============================] - 3s 9ms/step - loss: 0.1161 - accuracy: 0.9782 - val_loss: 0.2442 - val_accuracy: 0.9342 - lr: 0.0010
Epoch 5/10
333/333 [==============================] - 3s 10ms/step - loss: 0.0721 - accuracy: 0.9873 - val_loss: 0.2517 - val_accuracy: 0.9342 - lr: 0.0010
Epoch 6/10
333/333 [==============================] - 3s 8ms/step - loss: 0.0579 - accuracy: 0.9883 - val_loss: 0.2813 - val_accuracy: 0.9160 - lr: 0.0010
Epoch 7/10
333/333 [==============================] - 3s 8ms/step -

In [ ]:
model.save("model_squeeze_excite_resnet")

In [ ]:
!zip -r model_squeeze_excite_resnet.zip model_squeeze_excite_resnet/

  adding: model_squeeze_excite_resnet/ (stored 0%)
  adding: model_squeeze_excite_resnet/fingerprint.pb (stored 0%)
  adding: model_squeeze_excite_resnet/variables/ (stored 0%)
  adding: model_squeeze_excite_resnet/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: model_squeeze_excite_resnet/variables/variables.index (deflated 68%)
  adding: model_squeeze_excite_resnet/keras_metadata.pb (deflated 93%)
  adding: model_squeeze_excite_resnet/assets/ (stored 0%)
  adding: model_squeeze_excite_resnet/saved_model.pb (deflated 90%)
